In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Importing Libraries

In [ ]:
# Import necessary libraries
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

print("Libraries imported successfully!")

# 2. Dataset Preperation

##  2.1 Classification

In [ ]:
# Define paths for dataset
source_dir = "/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages"  
target_dir = "/kaggle/working/FilteredImages"  

# Select classes and limit images per class
selected_classes = ["collie", "dolphin", "elephant", "fox", "moose", "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"]
images_per_class = 650

# Create target directory and filter images
os.makedirs(target_dir, exist_ok=True)
for class_name in selected_classes:
    class_path = os.path.join(source_dir, class_name)
    target_path = os.path.join(target_dir, class_name)
    os.makedirs(target_path, exist_ok=True)
    for i, file_name in enumerate(os.listdir(class_path)):
        if i >= images_per_class:
            break
        full_file_name = os.path.join(class_path, file_name)
        if os.path.isfile(full_file_name):
            cv2.imwrite(os.path.join(target_path, file_name), cv2.imread(full_file_name))

print("Dataset prepared and balanced.")

##  2.2 Sizing and Normalization of Images

In [ ]:
# Function to load and process images
def load_and_process_images(data_dir, image_size=(128, 128)):
    images = []
    labels = []
    for class_name in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_name)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                file_path = os.path.join(class_path, file_name)
                img = cv2.imread(file_path)
                if img is not None:
                    img_resized = cv2.resize(img, image_size)
                    img_normalized = img_resized / 255.0
                    images.append(img_normalized)
                    labels.append(class_name)
    return np.array(images), np.array(labels)

# Load and process images
data_dir = target_dir
X, y = load_and_process_images(data_dir)
print(f"Dataset size: {X.shape}")

# 3. Data Augmentation

In [ ]:
# Define data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X)

print("Data augmentation applied.")

# 4. Creating the CNN Model

In [ ]:
# Create a simple CNN model
from tensorflow.keras.layers import Input

model = Sequential([
    Input(shape=(128, 128, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(selected_classes), activation='softmax')
])

print("CNN model created.")

#  5. Model Compilation and Training

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Encode string labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # String labels to integers

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Convert integer labels to one-hot encoding
num_classes = len(np.unique(y_encoded))  # Total number of classes
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

# Create TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_one_hot)).batch(32).shuffle(buffer_size=1024)
validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test_one_hot)).batch(32)

# Compile the model
model.compile(
    optimizer='adam',               # Optimization algorithm
    loss='categorical_crossentropy', # Loss function
    metrics=['accuracy']            # Evaluation metric
)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=20
)

print("Model training completed.")

# 6. Evaluating Model Performance

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(validation_dataset)
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test Loss: {test_loss:.2f}")

# 7. Model Predictions

In [ ]:
# Predict on test data
import numpy as np

# Get one batch from the test dataset
for images, labels in validation_dataset.take(1):
    predictions = model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(labels.numpy(), axis=1)

    # Visualize the results
    plt.figure(figsize=(12, 12))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"Predicted: {label_encoder.inverse_transform([predicted_classes[i]])[0]}\nTrue: {label_encoder.inverse_transform([true_classes[i]])[0]}")
        plt.axis("off")
    plt.show()
    break

# 8. Manipulated Test Set

In [ ]:
# Function to manipulate images
def manipulate_images(images):
    manipulated_images = []
    for img in images:
        manipulated = cv2.convertScaleAbs(img, alpha=1.5, beta=30)  # Increase brightness
        manipulated_images.append(manipulated)
    return np.array(manipulated_images)

# Apply manipulation to the test set
X_test_manipulated = manipulate_images(X_test)
manipulated_loss, manipulated_accuracy = model.evaluate(X_test_manipulated, y_test)
print(f"Manipulated Test Accuracy: {manipulated_accuracy * 100:.2f}%")

# 9. Gray World Algorithm for Color Constancy


In [ ]:
# Apply gray world algorithm for color constancy
def apply_gray_world(image):
    avg_b, avg_g, avg_r = cv2.mean(image)[:3]
    gray_value = (avg_b + avg_g + avg_r) / 3
    scaling_factors = np.array([gray_value / avg_b, gray_value / avg_g, gray_value / avg_r])
    corrected_image = image * scaling_factors
    corrected_image = np.clip(corrected_image, 0, 255).astype(np.uint8)
    return corrected_image

# Apply color constancy to manipulated test set
X_test_corrected = np.array([apply_gray_world(img) for img in X_test_manipulated])

# Evaluate model with color-corrected test set
corrected_loss, corrected_accuracy = model.evaluate(X_test_corrected, y_test)
print(f"Color Corrected Test Accuracy: {corrected_accuracy * 100:.2f}%")

# 10. Comparison of Results

In [ ]:
# Compare results
print("Results Comparison:")
print(f"Original Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Manipulated Test Accuracy: {manipulated_accuracy * 100:.2f}%")
print(f"Color Corrected Test Accuracy: {corrected_accuracy * 100:.2f}%")

# 11. Creating README File


In [ ]:
# Create README file
readme_content = """
# Image Classification Project

This project uses a subset of the Animals with Attributes 2 dataset to train a Convolutional Neural Network (CNN) for image classification. Key steps include:

1. Preparing the dataset by selecting specific classes and balancing the number of images per class.
2. Resizing and normalizing images to ensure consistency.
3. Applying data augmentation techniques to enhance model robustness.
4. Training a simple CNN model using TensorFlow/Keras.
5. Evaluating the model on original, manipulated, and color-corrected test sets.

## Dataset
- Source: [Animals with Attributes 2](https://datasets.d2.mpi-inf.mpg.de/awa2/)

## Steps
1. Data Preparation
2. Image Preprocessing
3. Data Augmentation
4. Model Training
5. Evaluation and Analysis

## Results
The model's accuracy is compared across original, manipulated, and color-corrected datasets to assess its performance under different conditions.

"""

# Save README to file
with open("/kaggle/working/README.md", "w") as file:
    file.write(readme_content)

print("README file created.")

## Vaktim kalmadigi icin istedigim sekilde sonuclayamadim, daha sonra duzeltecegim. 